# Домашнее задание №2
В данном задании предлагается решить [контест](https://www.kaggle.com/competitions/vk-edu-bmstu-asr-competition/overview) на платформе Kaggle.
Основная цель данного задания состоит в том, чтобы студенты ознакомились с зоопарком открытых моделей для решения задачи распознавания речи, а так же попрактиковались в подходах по постпроцессингу распознанных гипотез для улучшения качества ASR.

Разрешается пользоваться всеми предобученными моделями.

**Строго запрещено размечать данные вручную!**

## Загрузка данных и инициализация модели

In [ ]:
# https://drive.google.com/file/d/1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG/view?usp=sharing # ссылка для ручной загрузки данных

In [2]:
!gdown 1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG && tar -xf kaggle.tar

Downloading...
From (original): https://drive.google.com/uc?id=1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG
From (redirected): https://drive.google.com/uc?id=1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG&confirm=t&uuid=f0de545a-6992-42d8-853d-d23e2d743fb1
To: /content/kaggle.tar
100% 138M/138M [00:04<00:00, 29.1MB/s]


In [3]:
!ls -l kaggle/wav | wc -l

643


Для базового решения задания воспользуемся предобученной моделью Whisper от OpenAI. Для этого обратимся к платформе [HuggingFace](https://huggingface.co/collections/openai/whisper-release-6501bba2cf999715fd953013)

In [4]:
import os
import soundfile as sf
import torch
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import string

In [5]:
# загрузим модель
from transformers import WhisperProcessor, WhisperForConditionalGeneration

device = "cuda:0"
model_name = "openai/whisper-base"
processor = WhisperProcessor.from_pretrained(model_name)
processor.feature_extractor.return_attention_mask = True
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

## Baseline подход к инференсу модели

In [6]:
def batch_inference(model, processor, path_to_wavs, batch_size, sampling_rate=16000):
    results = {}
    wav_files = os.listdir(path_to_wavs)

    forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe") # generates task specific special tokens

    for i in tqdm(range(0, len(wav_files), batch_size), total=np.ceil(len(wav_files) / batch_size)):
        audio_paths = wav_files[i : i + batch_size]

        batch = []

        for path in audio_paths:
            audio, _ = sf.read(os.path.join(path_to_wavs, path))
            batch.append(audio)
        inputs = processor(batch, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

        x, x_masks = inputs["input_features"].to(device), inputs["attention_mask"].to(device)

        with torch.no_grad():
            model.eval()
            output_ids = model.generate(x, forced_decoder_ids=forced_decoder_ids, attention_mask=x_masks)

        transcribtion = processor.batch_decode(output_ids, skip_special_tokens=True)

        results.update(zip(audio_paths, transcribtion))
    return results

In [7]:
path_to_wavs = "kaggle/wav"
batch_size = 64

In [8]:
results = batch_inference(model=model, processor=processor, path_to_wavs=path_to_wavs, batch_size=batch_size)

100%|██████████| 11/11.0 [02:00<00:00, 10.94s/it]


## Sample submission
Подготовим пример для загрузки на платформу

In [9]:
def dummy_postprocessing(data):
    for filename, hypo in data.items():
        hypo = hypo.strip()
        hypo = hypo.translate(str.maketrans('', '', string.punctuation))
        hypo = hypo.lower()
        data[filename] = hypo
    return data

In [10]:
clean_data = dummy_postprocessing(results)

In [27]:
sample_submission = pd.read_csv("kaggle/sample_submission.csv")

In [28]:
sample_submission.head(20)

,filename
0,audio_0.wav
1,audio_1.wav
2,audio_2.wav
3,audio_3.wav
4,audio_4.wav
5,audio_5.wav
6,audio_6.wav
7,audio_7.wav
8,audio_8.wav
9,audio_9.wav


In [29]:
sample_submission["hypo"] = sample_submission["filename"].apply(lambda x: clean_data[x])

In [30]:
sample_submission.insert(0, "id", sample_submission.index)

In [33]:
sample_submission.head()

,id,filename,hypo
0,0,audio_0.wav,тревожная музыка
1,1,audio_1.wav,смех
2,2,audio_2.wav,батиша жрел ракта грямли сюда иди я сказал
3,3,audio_3.wav,высоль не понял
4,4,audio_4.wav,быстро за мной


In [32]:
sample_submission.to_csv("baseline.csv", index=False)

## Идеи по улучшению предсказаний:
* Всегда ли LAS архитектура работает на тишине корректно?
* Как насчет нормализации текста? (19.02 - девятнадцатое февраля, 20 лет - двадцать лет)
* Может быть попробовать рескоринг? (возвращаем больше, чем топ-1 гипотезу из бима)
* Может быть взять модель побольше?
## Соревнование доступно по [ссылке](https://www.kaggle.com/t/325b553468db4d01a583e29624a7685d)